# Лабораторная работа №3

Деревья решений. Ансамбли решающих деревьев.

## 0\. Импортировать необходимые модули и задать вспомогательные функции:

In [ ]:
%matplotlib inline

from collections.abc import Iterator
from contextlib import contextmanager
from datetime import timedelta
from time import perf_counter

import numpy as np
import pandas as pd
from matplotlib import pyplot as plt
from numpy.random import RandomState
from sklearn.ensemble import GradientBoostingClassifier, RandomForestClassifier
from sklearn.model_selection import KFold
from sklearn.preprocessing import LabelEncoder
from sklearn.tree import DecisionTreeClassifier, plot_tree
from sklearn.utils import shuffle

@contextmanager
def measure(message: str) -> Iterator[None]:
    start = perf_counter()
    yield
    end = perf_counter()
    display(f"Время, затраченное на {message}: {timedelta(seconds=(end - start))}")

### 1\.1\. Выбрать подходящую таблицу данных.

In [ ]:
adults: pd.DataFrame = pd.read_csv("https://archive.ics.uci.edu/ml/machine-learning-databases/adult/adult.data", header=None)

adults

In [ ]:
random = RandomState(0)

encoder = LabelEncoder()
encoder.fit(adults[14])

adults_X: pd.DataFrame = adults[[2, 4, 10, 11, 12]]
adults_Y = pd.Series(encoder.transform(adults[14]))

adults_X, adults_Y = shuffle(adults_X, adults_Y, random_state=random)

In [ ]:
adults_X

In [ ]:
adults_Y

### 1\.2\. Построить и визуализировать дерево решений.

### 3\.1\. Построить зависимость качества решения от числа вершин дерева.

In [ ]:
class Validator:
    def __init__(self, X: pd.DataFrame, Y: pd.Series, split_to: int = 5) -> None:  # noqa
        self._X = X
        self._Y = Y
        self._kf = KFold(split_to)

    def cross(self, model) -> np.array:
        result = []
        for idx, (training_slice, evaluation_slice) in enumerate(self._kf.split(self._X)):
            t_X: pd.DataFrame = self._X.iloc[training_slice]
            t_Y: pd.Series = self._Y.iloc[training_slice]
            e_X: pd.DataFrame = self._X.iloc[evaluation_slice]
            e_Y: pd.Series = self._Y.iloc[evaluation_slice]

            p_Y: np.ndarray = model.fit(t_X, t_Y).predict(e_X)
            result.append((p_Y == e_Y).sum() / e_Y.shape[0])
        return np.array(result)

    def training(self, model) -> np.array:
        result = []
        for idx, (training_slice, _) in enumerate(self._kf.split(self._X)):
            t_X: pd.DataFrame = self._X.iloc[training_slice]
            t_Y: pd.Series = self._Y.iloc[training_slice]

            p_Y: np.ndarray = model.fit(t_X, t_Y).predict(t_X)
            result.append((p_Y == t_Y).sum() / t_Y.shape[0])
        return np.array(result)

In [ ]:
validator = Validator(adults_X, adults_Y)

scores = []
for depth in range(1, 16):
    classifier = DecisionTreeClassifier(max_depth=depth, random_state=random)
    scores.append((depth, validator.cross(classifier).mean(axis=0)))

optimal_depth, optimal_score = max(scores, key=lambda s: s[1])
display(f"Оптимальная высота дерева на кросс-валидации: {optimal_depth}; счёт - {optimal_score}")
plt.plot(*zip(*scores), label="Кросс-валидация")

scores = []
for depth in range(1, 16):
    classifier = DecisionTreeClassifier(max_depth=depth, random_state=random)
    scores.append((depth, validator.training(classifier).mean(axis=0)))

optimal_depth, optimal_score = max(scores, key=lambda s: s[1])
display(f"Оптимальная высота дерева на тренировочных данных: {optimal_depth}; счёт - {optimal_score}")
plt.plot(*zip(*scores), label="Тренировочные данные")
plt.legend()

In [ ]:
can_be_drawn = 3
classifier = DecisionTreeClassifier(max_depth=can_be_drawn, random_state=random)
fitted = classifier.fit(adults_X, adults_Y)
plt.figure(figsize=(22, 8))
trees = plot_tree(fitted)

### 2\.1\. Применить метод градиентного бустинга.

In [ ]:
classifier = GradientBoostingClassifier(n_estimators=500, max_depth=3, random_state=random)

with measure("обучение модели"):
    fitted = classifier.fit(adults_X, adults_Y)

prediction: np.ndarray = fitted.predict(adults_X)
(prediction == adults_Y).sum() / adults_Y.shape[0]

### 2\.2\. Вычислить значимость параметров.

In [ ]:
importance = adults_X.columns.to_series().astype(float)
for idx, i in enumerate(classifier.feature_importances_):
    importance.iloc[idx] = i

importance

### 2\.3\. Выдать список построенных деревьев.

In [ ]:
estimator: np.ndarray
for estimator in classifier.estimators_[::100]:
    plt.figure(figsize=(22, 8))
    plot_tree(estimator.item())

### 4\.1\. Используя оценку на тренировочных данных, показать, как количество и глубина деревьев влияет на качество решения методом градиентного бустинга.

In [ ]:
DEPTH_LOWER_BOUND = 1
DEPTH_UPPER_BOUND = 2 + 1
DEPTH_STEP = 1

ESTIMATORS_N_LOWER_BOUND = 100
ESTIMATORS_N_UPPER_BOUND = 200 + 1
ESTIMATORS_N_STEP = 100

DEPTH_RANGE = range(DEPTH_LOWER_BOUND, DEPTH_UPPER_BOUND, DEPTH_STEP)
N_ESTIMATORS_RANGE = range(ESTIMATORS_N_LOWER_BOUND, ESTIMATORS_N_UPPER_BOUND, ESTIMATORS_N_STEP)

score_frame = lambda: pd.DataFrame(index=N_ESTIMATORS_RANGE, columns=DEPTH_RANGE, dtype=float)

In [ ]:
scores_gradient_training = score_frame()
for depth in DEPTH_RANGE:
    for n_estimators in N_ESTIMATORS_RANGE:
        classifier = GradientBoostingClassifier(n_estimators=n_estimators, max_depth=depth, random_state=random)
        scores_gradient_training[depth][n_estimators] = validator.training(classifier).mean(axis=0)

display(scores_gradient_training)
scores_gradient_training.plot()

### 4\.2\. Аналогично 4.1, но оценивать кросс-валидацией.

In [ ]:
scores_gradient_cross = score_frame()
for depth in DEPTH_RANGE:
    for n_estimators in N_ESTIMATORS_RANGE:
        classifier = GradientBoostingClassifier(n_estimators=n_estimators, max_depth=depth, random_state=random)
        scores_gradient_cross[depth][n_estimators] = validator.cross(classifier).mean(axis=0)

display(scores_gradient_cross)
scores_gradient_cross.plot()

### 5\.1\. Аналогично 4.1, но решать методом случайного леса.

In [ ]:
scores_random_forest_training = score_frame()
for depth in DEPTH_RANGE:
    for n_estimators in N_ESTIMATORS_RANGE:
        classifier = RandomForestClassifier(n_estimators=n_estimators, max_depth=depth, random_state=random)
        scores_random_forest_training[depth][n_estimators] = validator.training(classifier).mean(axis=0)

display(scores_random_forest_training)
scores_random_forest_training.plot()

### 5\.2\. Аналогично 4.2, но решать методом случайного леса.

In [ ]:
scores_random_forest_cross = score_frame()
for depth in DEPTH_RANGE:
    for n_estimators in N_ESTIMATORS_RANGE:
        classifier = RandomForestClassifier(n_estimators=n_estimators, max_depth=depth, random_state=random)
        scores_random_forest_cross[depth][n_estimators] = validator.cross(classifier).mean(axis=0)

display(scores_random_forest_cross)
scores_random_forest_cross.plot()

In [ ]:
scores_gradient_training